In [12]:
import numpy as np
import pandas as pd
import os
from importlib import reload
import find_cpt
from rgf.sklearn import RGFClassifier
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
data_dir = "./data"

In [3]:
test = pd.read_csv('./data/2017-01-01.csv')
all_features = list(test.loc[test['model'] == 'ST4000DM000'].dropna(axis=1, how='all').columns.values)
print(all_features)
def process_SgtA(df, name):
    df = df.loc[df['model'] == name]
    return df[all_features]
type_dict = {feature: np.float32 for feature in all_features[5:]}
SgtA = pd.concat([process_SgtA(pd.read_csv(os.path.join(data_dir, filename), dtype=type_dict), 'ST4000DM000') for filename in os.listdir(data_dir)])

['date', 'serial_number', 'model', 'capacity_bytes', 'failure', 'smart_1_normalized', 'smart_1_raw', 'smart_3_normalized', 'smart_3_raw', 'smart_4_normalized', 'smart_4_raw', 'smart_5_normalized', 'smart_5_raw', 'smart_7_normalized', 'smart_7_raw', 'smart_9_normalized', 'smart_9_raw', 'smart_10_normalized', 'smart_10_raw', 'smart_12_normalized', 'smart_12_raw', 'smart_183_normalized', 'smart_183_raw', 'smart_184_normalized', 'smart_184_raw', 'smart_187_normalized', 'smart_187_raw', 'smart_188_normalized', 'smart_188_raw', 'smart_189_normalized', 'smart_189_raw', 'smart_190_normalized', 'smart_190_raw', 'smart_191_normalized', 'smart_191_raw', 'smart_192_normalized', 'smart_192_raw', 'smart_193_normalized', 'smart_193_raw', 'smart_194_normalized', 'smart_194_raw', 'smart_197_normalized', 'smart_197_raw', 'smart_198_normalized', 'smart_198_raw', 'smart_199_normalized', 'smart_199_raw', 'smart_240_normalized', 'smart_240_raw', 'smart_241_normalized', 'smart_241_raw', 'smart_242_normalized

In [4]:
# find names of the failure disk
fail_names = SgtA.loc[SgtA['failure'] == 1]['serial_number'].unique()
print(fail_names.size)
print(fail_names.size / SgtA['serial_number'].unique().size)

1236
0.035104660740151665


In [5]:
reload(find_cpt)
# find failure disks
failure_disk_group = SgtA.loc[SgtA['serial_number'].isin(fail_names)].sort_values('date', ascending=True).groupby('serial_number')

def get_cpt(data):
    changepoint = find_cpt.cpt(data=data, type='normal-mean').find_changepoint()
    if changepoint > 0:
        return data.size - changepoint
    return changepoint

functions_group = {n: get_cpt for n in all_features[5:]}
all_cpt_series = failure_disk_group.agg(functions_group)
print(all_cpt_series)

               smart_1_normalized  smart_1_raw  smart_3_normalized  \
serial_number                                                        
S30070F5                     -1.0         -1.0                -1.0   
S300CCWE                     62.0         -1.0                -1.0   
S300PFN5                     -1.0        123.0                31.0   
S300RS9T                      1.0         -1.0                -1.0   
S300TQZA                     -1.0         -1.0                -1.0   
S300V3AD                     -1.0          5.0                -1.0   
S300VKP4                     93.0         -1.0                -1.0   
S300VL64                      7.0         -1.0               222.0   
S300VL6S                     -1.0         -1.0               221.0   
S300VLLF                     -1.0         -1.0                -1.0   
S300WDBK                     -1.0         -1.0                -1.0   
S300WE57                      1.0         41.0                -1.0   
S300WEE9            

In [ ]:
def get_percent(data):
    return data[(data>0) & (data <=100)].dropna().size/data.dropna().size
def get_median(data):
    return data[data>0].dropna().median()
def get_mean(data):
    return data[data>0].dropna().mean()
summarize = all_cpt_series.agg([get_percent, get_median, get_mean])
print(summarize)
summarize.to_csv('./preprocess/summarize.csv')

In [7]:
selected_features = summarize.loc['get_percent'].T.sort_values(ascending=False)
selected_features = selected_features[selected_features>0.01].index
print(selected_features)

Index(['smart_242_raw', 'smart_7_raw', 'smart_193_raw', 'smart_9_raw',
       'smart_240_raw', 'smart_190_normalized', 'smart_190_raw',
       'smart_194_raw', 'smart_194_normalized', 'smart_9_normalized',
       'smart_198_raw', 'smart_197_raw', 'smart_241_raw', 'smart_7_normalized',
       'smart_187_normalized', 'smart_187_raw', 'smart_4_raw', 'smart_12_raw',
       'smart_1_normalized', 'smart_193_normalized', 'smart_1_raw',
       'smart_5_raw', 'smart_3_normalized', 'smart_183_raw',
       'smart_183_normalized', 'smart_198_normalized', 'smart_197_normalized',
       'smart_5_normalized', 'smart_192_raw', 'smart_189_raw',
       'smart_189_normalized', 'smart_184_normalized', 'smart_184_raw'],
      dtype='object')


In [8]:
Sgt_features = ['serial_number', 'date', 'smart_1_normalized', 'smart_1_raw', 'smart_5_normalized', 'smart_5_raw', 'smart_7_normalized', 'smart_7_raw',
    'smart_184_normalized', 'smart_184_raw', 'smart_187_normalized', 'smart_187_raw', 'smart_188_raw', 'smart_189_normalized', 'smart_189_raw', 
    'smart_190_normalized', 'smart_190_raw', 'smart_193_normalized', 'smart_193_raw', 'smart_194_normalized', 'smart_194_raw', 'smart_197_normalized', 
    'smart_197_raw', 'smart_198_normalized', 'smart_198_raw', 'smart_240_raw', 'smart_241_raw', 'smart_242_raw', 'failure']
new_features = [i for i in selected_features if i not in Sgt_features]
print(new_features)

['smart_9_raw', 'smart_9_normalized', 'smart_4_raw', 'smart_12_raw', 'smart_3_normalized', 'smart_183_raw', 'smart_183_normalized', 'smart_192_raw']


In [9]:
print(summarize[new_features].loc['get_percent'])
# 3: Spin-Up Time (NA)
# 4: Start/Stop Count (not in)
# 9: Power-On Hours (not in)
# 12: Power Cycle Count (not in)
# 183: SATA Downshift Error Count or Runtime Bad Block (0.5%)
# 192: Power-off Retract Count, Emergency Retract Cycle Count (not in)

smart_9_raw             0.505663
smart_9_normalized      0.467638
smart_4_raw             0.308252
smart_12_raw            0.307443
smart_3_normalized      0.135113
smart_183_raw           0.127832
smart_183_normalized    0.127832
smart_192_raw           0.045307
Name: get_percent, dtype: float64


In [ ]:
# compact info.
def get_cmpt_info(data):
    return pd.ewma(data.values, span=np.round(summarize.loc['get_median', data.name]))[-1]
functions_group1 = {n: get_cmpt_info for n in selected_features.values}
# print(functions_group1)
compacted_info = SgtA.groupby('serial_number', as_index=False).agg(functions_group1)
print(compacted_info)
compacted_info.to_csv('./preprocess/compacted.csv')

In [17]:
# kmeans
# prepare set
X_health = compacted_info.loc[~compacted_info['serial_number'].isin(fail_names)].drop('serial_number', axis=1).values
kmeans = KMeans(n_clusters=150, random_state=0, n_jobs=-1).fit(X_health)

In [20]:
X_health_transformed = []
for j in range(0, 150):
    d = kmeans.transform(X_health)[:, j]
    ind = np.argsort(d)[::-1][:10]
    X_health_transformed[0:0] = list(X_health[ind])

In [21]:
X_failed = list(compacted_info.loc[compacted_info['serial_number'].isin(fail_names)].drop('serial_number', axis=1).values)
X = X_health_transformed + X_failed
y = np.concatenate((np.zeros(len(X_health_transformed)), np.ones(len(X_failed))), axis=0) 

In [22]:
def model_stat(model, X, y):
    f_score = np.average(cross_val_score(model, X, y, cv=5, scoring='f1', n_jobs=-1))
    r_score = np.average(cross_val_score(model, X, y, cv=5, scoring='recall', n_jobs=-1))
    p_score = np.average(cross_val_score(model, X, y, cv=5, scoring='precision', n_jobs=-1))
    print(f_score)
    print(r_score)
    print(p_score)

In [93]:
# tune LR
lr_model = LogisticRegressionCV(Cs=100, fit_intercept=True, cv=5, 
                                        dual=False, penalty='l2', scoring='f1', 
                                        solver='newton-cg',  max_iter=1000, class_weight='balanced',
                                        n_jobs=-1, refit=True, multi_class='ovr', random_state=0, verbose=1)
lr_model.fit(X, y)
model_stat(lr_model, X, y)

[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    4.2s remaining:    6.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    7.1s finished
C:\Users\Ray\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:461: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Ray\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Ray\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


0.8973881364942604
0.8697531670366985
0.9475660415406331


In [46]:
# tune RF
n_estimators = [5] # tuned
max_features = ['auto'] # tuned
criterion = ['entropy']  # tuned
max_depth = [12] # tuned
min_samples_split = [2]
min_samples_leaf = [1]
min_weight_fraction_leaf = [0]
max_leaf_nodes = [None]

bootstrap = [False]
random_state = [0]
class_weight = ['balanced']
search_grid = {
    'n_estimators': n_estimators,
    'criterion': criterion,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'min_weight_fraction_leaf': min_weight_fraction_leaf,
    'max_leaf_nodes': max_leaf_nodes,
    'bootstrap': bootstrap,
    'random_state': random_state,
    'class_weight': class_weight
    }

rf_model = RandomForestClassifier()
rf_grid = GridSearchCV(estimator=rf_model, param_grid=search_grid, 
    cv=5, scoring='f1', n_jobs=-1, verbose=2)
rf_grid.fit(X, y)

model_stat(rf_model, X, y)
print(rf_grid.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.5s remaining:    2.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.3s finished


0.9858837068178783
0.9927158155935745
0.9802919708029197
{'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 12, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0, 'n_estimators': 5, 'random_state': 0}


In [41]:
# tune SVM

search_grid = {
    'C': [1.05], 
    'kernel': ['rbf'],
    'gamma': [0.05],
    'class_weight': ['balanced'],
    'max_iter': [-1],
    'random_state': [0]
}
svm_model = SVC()
svm_grid = GridSearchCV(estimator=svm_model, param_grid=search_grid, 
    cv=5, scoring='f1', n_jobs=-1, verbose=2)
svm_grid.fit(X, y)
model_stat(svm_model, X, y)
print(svm_grid.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    2.0s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.7s finished


0.9896353166986565
1.0
0.9802919708029197
{'C': 1.05, 'class_weight': 'balanced', 'gamma': 0.05, 'kernel': 'rbf', 'max_iter': -1, 'random_state': 0}


In [80]:
# tune GBDT
search_grid = {
    'n_estimators': [150],
    'min_samples_split': [2],
    'min_samples_leaf': [85],
    'max_depth': [8],
    'max_features':['sqrt'],
    'subsample': [0.8],
    'random_state': [0]
}
gbdt_model = GradientBoostingClassifier()
gbdt_grid = GridSearchCV(estimator= gbdt_model, param_grid=search_grid, 
    cv=5, scoring='f1', n_jobs=-1, verbose=2)
gbdt_grid.fit(X, y)
model_stat(gbdt_model, X, y)
print(gbdt_grid.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.7s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.5s finished


0.9902997550486601
0.9862446127726263
0.99296875
{'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 85, 'min_samples_split': 2, 'n_estimators': 150, 'random_state': 0, 'subsample': 0.8}


In [89]:
# tune DT (TBI)
search_grid = {
    'criterion': ['entropy'],
    'min_samples_split': [281],
    'min_samples_leaf': [50],
    'max_depth': [8],
    'max_features':['sqrt'],
    'random_state': [0]
}
dt_model = DecisionTreeClassifier()
dt_grid = GridSearchCV(estimator= dt_model, param_grid=search_grid, 
    cv=5, scoring='f1', n_jobs=-1, verbose=2)
dt_grid.fit(X, y)
model_stat(dt_model, X, y)
print(dt_grid.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.3s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.9s finished


0.9907036730467572
0.9894802141831004
0.99296875
{'criterion': 'entropy', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 50, 'min_samples_split': 281, 'random_state': 0}


In [102]:
# tune RGF
search_grid = {
    'max_leaf': [1000],
    'algorithm': ['RGF_Sib'],
    'test_interval': [100],
    'loss': ['Log']
}
# loss: You can select "LS", "Log", "Expo" or "Abs".
rgf_model = RGFClassifier()
rgf_grid = GridSearchCV(estimator= rgf_model, param_grid=search_grid, 
    cv=5, scoring='f1', n_jobs=-1, verbose=2)
rgf_grid.fit(X, y)
model_stat(rgf_model, X, y)
print(rgf_grid.best_params_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    6.1s remaining:    2.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    6.5s finished


0.9887500787474928
0.987864045971007
0.99
{'algorithm': 'RGF_Sib', 'loss': 'Log', 'max_leaf': 1000, 'normalize': False, 'test_interval': 100}
